In [24]:
import pandas as pd
import numpy as np
import matplotlib as plt
import openpyxl

data = pd.read_excel('FR2209_assessment_data.xlsx', index_col=0, header=0)
data = data.iloc[:121,:]

stocks = data.loc[:,['BYD','DNB','IAG','SAM','CHM','X']]

returns = stocks.pct_change()
std = returns.std()
mean = returns.mean()


returns



,BYD,DNB,IAG,SAM,CHM,X
date,,,,,,
2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2013-02-28,-0.062767,0.065856,0.075332,0.085985,0.065668,-0.065486
2013-03-29,0.258752,-0.017578,0.060728,0.055986,0.109166,-0.064299
2013-04-30,0.451028,0.115651,0.098254,-0.015033,-0.026455,-0.087180
2013-05-31,0.085833,0.018199,0.005871,0.059864,0.024632,-0.003596
...,...,...,...,...,...,...
2022-08-31,-0.019456,-0.092640,-0.129345,0.170433,-0.068707,-0.030908
2022-09-30,-0.121827,-0.130526,-0.159940,-0.011528,-0.170479,-0.207696
2022-10-31,0.212171,0.037127,0.319528,-0.112438,-0.085301,0.123621


In [25]:
#asssume the risk free of 0.5%, calculate the excess return for each stock
R_f = 0.05
excessret = mean - R_f /12
excessret

BYD    0.582069
DNB    0.398071
IAG    0.257447
SAM    0.248595
CHM    0.197914
X      0.189037
dtype: float64

In [26]:
#sharpe for each stock
sharpe = excessret* np.sqrt(3620)/ returns.std()
sharpe

BYD    5.646475
DNB    5.236055
IAG    8.003445
SAM    6.534485
CHM    6.619993
X      5.714190
dtype: float64

In [27]:
#returns variance / covariance matrix 
returns.cov()

,BYD,DNB,IAG,SAM,CHM,X
BYD,38.468336,28.357208,7.463509,-0.101051,-0.034629,12.280237
DNB,28.357208,20.922912,5.493531,-0.077406,-0.023190,9.055468
IAG,7.463509,5.493531,3.745670,-0.012002,-0.036343,2.407210
SAM,-0.101051,-0.077406,-0.012002,5.239277,0.368444,0.004649
CHM,-0.034629,-0.023190,-0.036343,0.368444,3.235537,-0.010240
X,12.280237,9.055468,2.407210,0.004649,-0.010240,3.961793


In [39]:
import numpy as np
from itertools import combinations, permutations, product

# Generate permutations of weights in multiples of 0.1
permutations = product(range(11), repeat=len(mean))

# Filter permutations where the sum equals 10 (1 in multiples of 0.1)
valid_permutations = [p for p in permutations if sum(p) == 10]

# Convert permutations to multiples of 0.1 and normalize each permutation
normalized_permutations = [[weight / 10 for weight in p] for p in valid_permutations]

print(normalized_permutations)

[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.1, 0.9], [0.0, 0.0, 0.0, 0.0, 0.2, 0.8], [0.0, 0.0, 0.0, 0.0, 0.3, 0.7], [0.0, 0.0, 0.0, 0.0, 0.4, 0.6], [0.0, 0.0, 0.0, 0.0, 0.5, 0.5], [0.0, 0.0, 0.0, 0.0, 0.6, 0.4], [0.0, 0.0, 0.0, 0.0, 0.7, 0.3], [0.0, 0.0, 0.0, 0.0, 0.8, 0.2], [0.0, 0.0, 0.0, 0.0, 0.9, 0.1], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.1, 0.0, 0.9], [0.0, 0.0, 0.0, 0.1, 0.1, 0.8], [0.0, 0.0, 0.0, 0.1, 0.2, 0.7], [0.0, 0.0, 0.0, 0.1, 0.3, 0.6], [0.0, 0.0, 0.0, 0.1, 0.4, 0.5], [0.0, 0.0, 0.0, 0.1, 0.5, 0.4], [0.0, 0.0, 0.0, 0.1, 0.6, 0.3], [0.0, 0.0, 0.0, 0.1, 0.7, 0.2], [0.0, 0.0, 0.0, 0.1, 0.8, 0.1], [0.0, 0.0, 0.0, 0.1, 0.9, 0.0], [0.0, 0.0, 0.0, 0.2, 0.0, 0.8], [0.0, 0.0, 0.0, 0.2, 0.1, 0.7], [0.0, 0.0, 0.0, 0.2, 0.2, 0.6], [0.0, 0.0, 0.0, 0.2, 0.3, 0.5], [0.0, 0.0, 0.0, 0.2, 0.4, 0.4], [0.0, 0.0, 0.0, 0.2, 0.5, 0.3], [0.0, 0.0, 0.0, 0.2, 0.6, 0.2], [0.0, 0.0, 0.0, 0.2, 0.7, 0.1], [0.0, 0.0, 0.0, 0.2, 0.8, 0.0], [0.0, 0.0, 0.0, 0.3, 0.0, 0.7], [0.0, 0